### <center>2018 Winter CS101.05</center>

# <center>基于数组的序列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 课后作业-01

- 今日截止
- 习题讲解

## 今日议程

1. Python的序列类型
2. 底层数组结构
3. 动态数组和摊销
4. Python序列类型的效率
5. 多维数组

## Python的序列类型

Python内置了各种“序列”类，这些类底层都使用了类似数组的结构，它们都支持下标访问，如seq[k]：

- 列表(list)
- 元祖(tuple)
- 字符串(str)




本课程重点介绍序列的：

- 行为
- 实现细节
- 效率分析

## 底层数组结构

- 1byte = 8bit
- RAM(Ramdom Access Memory)
    - 认为存储器的任一单个字节被存储或检索的运行时间为O(1)
- 内存地址
- 广义上的数组结构：利用计算机储存器内的一块连续的存储空间，来存储一组具有相同类型的数据
    - 线性结构
    - 随机访问

![](./img/5-2.png)

数组根据下标随机访问的寻址公式：

> a[i]_address = base_address + i * data_type_size

### Python中的引用数组

Python内置的**list**和**tuple**类型底层使用一块连续的内存地址来存储对象的引用

![](./img/5-3.png)


In [1]:
primes = [2, 3, 5, 7, 11, 13, 17, 19]
temp = primes[3:6]

![](./img/5-4.png)

In [18]:
temp[2] = 15
print(temp)
print(primes)

[7, 11, 15]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]


![](./img/5-5.png)

In [7]:
counters = [0] * 8
counters

[0, 0, 0, 0, 0, 0, 0, 0]

![](./img/5-6.png)

In [8]:
counters[2] += 1
counters

[0, 0, 1, 0, 0, 0, 0, 0]

![](./img/5-7.png)

In [15]:
for i in range(len(counters)):
    print(id(counters[i]))

1654942752
1654942752
1654942784
1654942752
1654942752
1654942752
1654942752
1654942752


In [10]:
extras = [23, 29, 31]
primes.extend(extras)
primes

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]

![](img/5-8.png)

In [14]:
id(extras[0]) == id(primes[8])

True

### Python中的紧凑数组

- 不再像引用数组一样存储内存的地址，更高效利用内存空间
- 紧凑数组中的元素在内存里中是连续存放的
- array模块

In [2]:
from array import array
primes = array('i', [2, 3, 5, 7, 11, 13, 17, 19])
primes

array('i', [2, 3, 5, 7, 11, 13, 17, 19])

- array模块支持的类型

|参数|数据类型|字节数|
|---|---|---|
|'b'|signed char|1|
|'B'|unsigned char|1|
|'u'|Unicode char|2/4|
|'h'|signed short int|2|
|'H'|unsigned short int|2|
|'i'|signed int|2/4|
|'I'|unsigned int|2/4|
|'l'|signed long int|4|
|'L'|unsigned long int|4|
|'f'|float|4|
|'d'|double|8|

## 动态数组和均摊

Python的list类可以视为一种动态数组的实现

In [13]:
import sys
data = []
for k in range(10):
    l = len(data)
    s = sys.getsizeof(data)
    print('List length: {0:3d}; Size in bytes: {1:4d}'.format(l, s))
    data.append(k)

List length:   0; Size in bytes:   64
List length:   1; Size in bytes:   96
List length:   2; Size in bytes:   96
List length:   3; Size in bytes:   96
List length:   4; Size in bytes:   96
List length:   5; Size in bytes:  128
List length:   6; Size in bytes:  128
List length:   7; Size in bytes:  128
List length:   8; Size in bytes:  128
List length:   9; Size in bytes:  192


### 实现一个动态数组

当数组A的容量达到上限，需要插入新的元素时，扩展数组A的步骤如下：

1. 新分配一个更大的数组B；
2. 令B[i]=A[i] (i=0,1,2...,n-1)；
3. 令A=B, 在新分配的空间内添加元素；

![](img/5-9.png)

In [14]:
import ctypes


class DynamicArray:
    """A simplified Python list which provides low-level array.
    """

    def __init__(self):
        """Create an empty array when initialied.
        """
        self._len = 0
        self._cap = 1
        self._A = self._make_array(self._cap)

    def _make_array(self, cap):
        """Return new array with capacity.
        """
        return (cap * ctypes.py_object)()

    def __len__(self):
        """Return the number of elements in array.
        """
        return self._len

    def __getitem__(self, k):
        """Return the element of array which indexed at k.
        """
        if not 0 <= k <= self._len:
            raise IndexError("wrong index")
        return self._A[k]

    def append(self, element):
        """Add element to the end of array.
        """
        if self._len == self._cap:
            self._resize(2 * self._cap)
        self._A[self._len] = element
        self._len += 1

    def _resize(self, cap):
        B = self._make_array(cap)
        for i in range(self._len):
            B[i] = self._A[i]
        self._A = B
        self._cap = cap

In [18]:
da = DynamicArray()
for k in range(10):
    l = len(da)
    s = da._cap
    print('DynamicArray length: {0:3d}; capacity: {1:4d}'.format(l, s))
    da.append(k)

DynamicArray length:   0; capacity:    1
DynamicArray length:   1; capacity:    1
DynamicArray length:   2; capacity:    2
DynamicArray length:   3; capacity:    4
DynamicArray length:   4; capacity:    4
DynamicArray length:   5; capacity:    8
DynamicArray length:   6; capacity:    8
DynamicArray length:   7; capacity:    8
DynamicArray length:   8; capacity:    8
DynamicArray length:   9; capacity:   16


### 动态数组的均摊分析

![](img/5-10.png)

# Any Questions?